In [1]:
import csv, pandas as pd, numpy as np

In [93]:
def fixer(filename_list):
    
    df1 = pd.read_csv('../data/{0}'.format(filename_list[0]))
    if len(filename_list) > 1:
        for file in filename_list[1:]:
            new_df = pd.read_csv('../data/{0}'.format(file))
            df1 = pd.concat([df1, new_df])
    
    data = df1.reset_index()
    data = data.fillna(0)
    del data['index']
            
    mapper = pd.read_csv("../data/case_data.csv")
    
    del data['Unnamed: 0']
    data.set_index("county")
    
    data2 = pd.melt(data, id_vars=['county'])
    data3 = data2.rename(columns={'variable':'date', 'county':'FIPS', 'value':'cases'})
    data3['FIPS'] = data3['FIPS'].apply(lambda x: str(x) if len(str(x)) == 5 else str(x))
    data4 = data3
    mapper['FIPS'] = mapper['FIPS'].apply(lambda x: str(x).split(".")[0])
    data4 = data4.merge(mapper, on='FIPS')
    
    del data4['Country_Region']
    del data4['Last_Update']
    del data4['Lat']
    del data4['Long_']
    del data4['Confirmed']
    del data4['Deaths']
    del data4['Recovered']
    del data4['Active']
    del data4['Combined_Key']
    
    data5 = data4.sort_values('date')
    data5 = data5.rename(columns={'FIPS':'fips', 'Admin2':'county', 'Province_State':'state'})
    ir = filename_list[0].split("_")[2]
    rr = filename_list[0].split("_")[3]
    
    #get historical data from nyt
    historical_data = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv")
    del historical_data['deaths']
    #print(historical_data['state'].unique())
    #format fips correctly
    #historical_data['fips'] = historical_data['fips'].apply(lambda x: str(x) if len(str(x)) == 5 else "0"+str(x))
    historical_data['fips'] = historical_data['fips'].apply(lambda x: str(x).split(".")[0])
    #only include the fips ids we're looking at for this viz
    historical_data = historical_data.loc[historical_data['fips'].isin(list(data5['fips']))]
    historical_data.reset_index()
    indexNames = historical_data[ historical_data['fips'] == 'nan' ].index
    historical_data.drop(indexNames, inplace=True)
    #filter dates by after a date
    historical_data.sort_values('date')
    historical_data['date'] = pd.to_datetime(historical_data['date'])
    historical_data = historical_data[(historical_data["date"] >= '2020-04-18') & (historical_data["date"] <= '2020-04-21')]
    historical_data['date'] = historical_data['date'].apply(lambda x: str(x).split(" ")[0]) 
    #decrease cases by factor of 1000 in order to match our predicted data
    historical_data['cases'] = historical_data['cases'].apply(lambda x: x/1000)
    #compile together
    data6 = pd.concat([historical_data,data5], ignore_index=True)
    data6.sort_values(by=['date'])
    data6['fips'] = data6['fips'].apply(lambda x: "0" + x if len(x) < 5 else x)
    
    #add population data:
    population_df = pd.read_csv("../data/census_data.csv", encoding="ISO-8859-1")
    del population_df['SUMLEV']
    del population_df['REGION']
    del population_df['DIVISION']
    del population_df['STATE']
    del population_df['COUNTY']
    del population_df['CENSUS2010POP']
    pop = population_df.rename(columns={'STNAME':'state', 'CTYNAME':'county', 'POPESTIMATE2019':'population'})
    pop['county'] = pop['county'].str.split(" Count", expand=True)[0]
        
    data7 = data6.merge(pop, on=['state', 'county'])
    means = data7.groupby(['state'])['population'].mean()
    data7['population'] = data7['population'].fillna(means)
    data7['date'] = pd.to_datetime(data7['date'])
    data7 = data7.sort_values(by=['date', 'cases', 'fips'],ascending=True)
    print(data7)
    
    data7.to_csv("../data/combined_{0}_{1}".format(ir, rr))

In [97]:
tester_full = ["output_Connecticut_0.001_0.06.csv","output_Maine_0.001_0.06.csv", "output_Vermont_0.001_0.06.csv", "output_New Hampshire_0.001_0.06.csv", "output_Rhode Island_0.001_0.06.csv"]
tester_full2 = ["output_Connecticut_0.003_0.08.csv","output_Maine_0.003_0.08.csv", "output_Vermont_0.003_0.08.csv", "output_New Hampshire_0.003_0.08.csv", "output_Rhode Island_0.003_0.08.csv"]
tester_full3 = ["output_Connecticut_0.01_0.8.csv","output_Maine_0.01_0.8.csv", "output_Vermont_0.01_0.8.csv", "output_New Hampshire_0.01_0.8.csv", "output_Rhode Island_0.01_0.8.csv"]

fixer(tester_full3)

/Users/Elise/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


      cases      county       date   fips          state  population
420   0.001       Essex 2020-04-18  50009        Vermont        6163
90    0.002   Aroostook 2020-04-18  23003          Maine       67055
210   0.002  Washington 2020-04-18  23029          Maine       31379
260   0.002        Coos 2020-04-18  33007  New Hampshire       31563
440   0.003  Grand Isle 2020-04-18  50013        Vermont        7235
..      ...         ...        ...    ...            ...         ...
29   15.000  Litchfield 2020-04-27  09005    Connecticut      180333
109  20.000  Cumberland 2020-04-27  23005          Maine      295003
19   26.000    Hartford 2020-04-27  09003    Connecticut      891720
9    38.000   Fairfield 2020-04-27  09001    Connecticut      943332
369  41.000  Providence 2020-04-27  44007   Rhode Island      638931

[520 rows x 6 columns]


/Users/Elise/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [95]:
def total_combiner(filename_list):
    df1 = pd.read_csv('../data/{0}'.format(filename_list[0]))
    df1['rate'] = filename_list[0].split("_")[1] + "_" + filename_list[0].split("_")[2].split(".c")[0]
#     df1['rr'] = filename_list[0].split("_")[2].split(".c")[0]
    if len(filename_list) > 1:
        for file in filename_list[1:]:
            new_df = pd.read_csv('../data/{0}'.format(file))
            new_df['rate'] = file.split("_")[1] + "_" + file.split("_")[2].split(".c")[0]
            #new_df['rr'] = file.split("_")[2].split(".c")[0]
            df1 = pd.concat([df1, new_df])
    
    data = df1.reset_index()
    data = data.fillna(0)
    del data['index']
    del data['Unnamed: 0']

    data['fips'] = data['fips'].apply(lambda x: "0" + str(x) if len(str(x)) < 5 else str(x))
    data = data.sort_values(by=['date'])
    print(data)
    data.to_csv("../data/total_combined.csv")

In [98]:
total_combiner(['combined_0.001_0.06.csv', 'combined_0.003_0.08.csv', 'combined_0.01_0.8.csv'])

       cases        county        date   fips          state  population  \
0      0.001         Essex  2020-04-18  50009        Vermont        6163   
1062   0.035  Androscoggin  2020-04-18  23001          Maine      108277   
1061   0.034       Windsor  2020-04-18  50027        Vermont       55062   
1060   0.031    Washington  2020-04-18  50023        Vermont       58409   
1059   0.030       Carroll  2020-04-18  33003  New Hampshire       48910   
...      ...           ...         ...    ...            ...         ...   
1010  28.000       Orleans  2020-04-27  50019        Vermont       27037   
1011  33.000      Sullivan  2020-04-27  33019  New Hampshire       43146   
1012  34.000    Bennington  2020-04-27  50003        Vermont       35470   
988    0.000  Androscoggin  2020-04-27  23001          Maine      108277   
1559  41.000    Providence  2020-04-27  44007   Rhode Island      638931   

            rate  
0     0.001_0.06  
1062    0.01_0.8  
1061    0.01_0.8  
1060    0.0